## 1) install package

In [1]:
import cellpose

## 2) IMPORT PACKAGE

In [9]:
import argparse
from os import listdir
from os.path import isfile, join
import nd2
from pathlib import Path
import numpy as np
import tifffile
import cellpose
from cellpose import models
from matplotlib import pyplot as plt
from tqdm import tqdm
import bigfish.detection as detection
import bigfish.stack as stack
import pandas as pd

## 3 ) Split image channel

In [3]:

### Path to  folder where your images are
path_exp_folder  = "/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/"
### Path to folder where you want to save the channel 0
path_save_ch0 = "/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch0/"

### Path to folder where you want to save the channel 1
path_save_ch1 = "/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch1/"
### Path to folder where you want to save the channel 0 in 2D
path_save_ch0_mip = "/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch0_mip/"

### Path to folder where you want to save the channel 1 in 2D
path_save_ch1_mip = "/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch1_mip/"

Path(path_save_ch0).mkdir(parents=True, exist_ok=True)
Path(path_save_ch1).mkdir(parents=True, exist_ok=True)
Path(path_save_ch0_mip).mkdir(parents=True, exist_ok=True)
Path(path_save_ch1_mip).mkdir(parents=True, exist_ok=True)
for im_path in Path(path_exp_folder).glob(f'*.nd2'):
    my_array = nd2.imread(im_path)
    tifffile.imsave(path_save_ch0 + im_path.name[:-4] + '.tiff', my_array[:,0])
    tifffile.imsave(path_save_ch1 + im_path.name[:-4] + '.tiff', my_array[:,1])
    tifffile.imsave(path_save_ch0_mip + im_path.name[:-4] + '.tiff', np.amax(my_array[:,0], 0))
    tifffile.imsave(path_save_ch1_mip + im_path.name[:-4] + '.tiff', np.amax(my_array[:,1], 0))

<ipython-input-3-feddb6abdf3e>:20: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(path_save_ch0 + im_path.name[:-4] + '.tiff', my_array[:,0])
<ipython-input-3-feddb6abdf3e>:21: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(path_save_ch1 + im_path.name[:-4] + '.tiff', my_array[:,1])
<ipython-input-3-feddb6abdf3e>:22: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(path_save_ch0_mip + im_path.name[:-4] + '.tiff', np.amax(my_array[:,0], 0))
<ipython-input-3-feddb6abdf3e>:23: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(path_save_ch1_mip + im_path.name[:-4] + '.tiff', np.amax(my_array[:,1], 0))


## 4) cell segmentation

In [4]:

##################
## Function used for cell segmentation
##################


def stitch3D_z(masks, stitch_threshold=0.25):
    """ stitch 2D masks into 3D volume with stitch_threshold on IOU """
    mmax = masks[0].max()
    for i in range(len(masks)-1):
        try:
            iou = cellpose.metrics._intersection_over_union(masks[i+1], masks[i])[1:,1:]
            iou[iou < stitch_threshold] = 0.0
            iou[iou < iou.max(axis=0)] = 0.0
            istitch = iou.argmax(axis=1) + 1
            ino = np.nonzero(iou.max(axis=1)==0.0)[0]
            istitch[ino] = np.arange(mmax+1, mmax+len(ino)+1, 1, int)
            mmax += len(ino)
            istitch = np.append(np.array(0), istitch)
            masks[i+1] = istitch[masks[i+1]]
        except Exception as e:
            print(e)
            print("in stich")
            continue
    return masks



def erase_solitary(mask): #mask en 3D
    """
    Erase nuclei  that are present in only one Z-slice
    Args:
        mask ():

    Returns:

    """
    mask_bis = np.zeros(mask.shape)
    current_nuclei = set(np.unique(mask[0]))
    post_nuclei = set(np.unique(mask[1]))
    nuclei_to_remove =  current_nuclei - post_nuclei
    nuclei_to_keep = current_nuclei - nuclei_to_remove # reminder: set operation are different from arithemtic operation
    for nuc in nuclei_to_keep:
        mask_bis[0] += (mask[0] == nuc) * mask[0]

    for i in range(1, len(mask)-1):
        pre_nuclei = set(np.unique(mask[i-1]))
        current_nuclei = set(np.unique(mask[i]))
        post_nuclei = set(np.unique(mask[i+1]))
        nuclei_to_remove =  current_nuclei - pre_nuclei - post_nuclei
        nuclei_to_keep = current_nuclei - nuclei_to_remove # reminder: set operation are different from arithemtic operation
        for nuc in nuclei_to_keep:
            mask_bis[i] += (mask[i] == nuc) *  mask[i]
    ##traiter le cas ou n = -1
    current_nuclei = set(np.unique(mask[-1]))
    pre_nuclei = set(np.unique(mask[-2]))
    nuclei_to_remove =  current_nuclei - pre_nuclei
    nuclei_to_keep = current_nuclei - nuclei_to_remove # reminder: set operation are different from arithemtic operation
    for nuc in nuclei_to_keep:
        mask_bis[-1] += (mask[-1] == nuc) * mask[-1]
    return mask_bis




def erase_small_nuclei(mask, min_size = 340):
    for nuc in tqdm(np.unique(mask)[1:]): ## remove zero
        sum_size = np.sum((mask == nuc).astype(int))
        print(sum_size)
        if sum_size < min_size:
                mask[mask == nuc] = 0
    return mask


def segment_nuclei(path_to_dapi_folder,
                   regex_dapi,
                   path_to_mask_dapi,
                   dico_param,
                   model,
                   save=True,
                   ):


    """
    segment dapi image and save  them in th path_to_mask_dapi folder
    Args:
        path_to_dapi (str):
        path_to_mask_dapi (str):
        dico_param (dict):
        model (cellpose modem):
        save (bool):
    Returns:
        None
    """

    if path_to_mask_dapi[-1] != "/":
        path_to_mask_dapi += "/"
    print(list(Path(path_to_dapi_folder).glob(f"*{regex_dapi}*")))
    print(f'dico_param{dico_param}')
    for path_dapi in tqdm(list(Path(path_to_dapi_folder).glob(f"*{regex_dapi}*"))):
        path_dapi = str(path_dapi)
        print(path_dapi)
        img = tifffile.imread(path_dapi)
        print(img.shape)
        if dico_param["mip"] is True and len(img.shape) == 3:
            img = np.amax(img, 0)
        else:
            if len(img.shape) == 3:
                img = img.reshape(img.shape[0], 1, img.shape[1], img.shape[2])
                print(f'image dapi shape after reshape {img.shape}')
                img = list(img)
        masks, flows, styles, diams = model.eval(img, diameter=dico_param["diameter"],
                                                 channels=[0, 0],
                                                 flow_threshold=dico_param["flow_threshold"],
                                                 do_3D=dico_param["do_3D"],
                                                 stitch_threshold=0)

        if len(masks.shape) == 3:
            masks = stitch3D_z(masks, dico_param["stitch_threshold"])
            masks = np.array(masks, dtype = np.int16)
            if len(masks.shape) and dico_param["erase_solitary"]:
                masks = erase_solitary(masks)
        if dico_param["erase_small_nuclei"] is not None:
            print(f'erase_small_nuclei threshold {dico_param["erase_small_nuclei"]}')
            masks = erase_small_nuclei(masks)
        if save:
            image_name = path_dapi.split('/')[-1].split(f'_{regex_dapi}')[0]
            tifffile.imwrite(path_to_mask_dapi + image_name +'.tif', data=masks, dtype=masks.dtype)
            np.save(path_to_mask_dapi + "dico_param.npy", dico_param)


In [13]:
#### run segmentation

### folder with your dapi images (2D or 3D)
path_to_dapi_folder = path_exp_folder + "ch1_mip/"

## Folder where you save segmentation mask 
path_to_mask_dapi = path_exp_folder + "ch1_mask_mip/"


Path(path_to_mask_dapi).mkdir(parents=True, exist_ok=True)
model_name = "cyto"
model = models.Cellpose(model_type=model_name)
dico_param = {}
dico_param["diameter"] = 80
dico_param["flow_threshold"] = 0.8
dico_param["mask_threshold"] = 0
dico_param["do_3D"] = False
dico_param["mip"] = False
dico_param["projected_focused"] = False
dico_param["stitch_threshold"] = 0.3
dico_param["erase_solitary"] = False
dico_param["erase_small_nuclei"] = None

for i in range(20, 19, -10):
    print(i)
    dico_param["diameter"] = i

    path_to_mask_dapi_loop = path_to_mask_dapi + str(i) + "_"+ model_name + "/"
    Path(path_to_mask_dapi_loop).mkdir(parents=True, exist_ok=True)

    segment_nuclei(path_to_dapi_folder=path_to_dapi_folder,
                   regex_dapi="ti",
                   path_to_mask_dapi=path_to_mask_dapi_loop,
                   dico_param=dico_param,
                   model=model,
                   save=True,
                   )


20
[PosixPath('/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch1_mip/WT_pCenTy_002.tiff'), PosixPath('/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch1_mip/Delta_adk1_pCenTy_001.tiff'), PosixPath('/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch1_mip/Delta_adk1_pCenTy_003.tiff'), PosixPath('/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch1_mip/WT_pCenTy_001.tiff'), PosixPath('/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch1_mip/Delta_adk1_pCenTy_002.tiff')]
dico_param{'diameter': 20, 'flow_threshold': 0.8, 'mask_threshold': 0, 'do_3D': False, 'mip': False, 'projected_focused': False, 'stitch_threshold': 0.3, 'erase_solitary': False, 'erase_small_nuclei': None}


  0%|          | 0/5 [00:00<?, ?it/s]

/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch1_mip/WT_pCenTy_002.tiff
(2160, 2560)


 20%|██        | 1/5 [00:51<03:25, 51.27s/it]

/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch1_mip/Delta_adk1_pCenTy_001.tiff
(2160, 2560)


...

 60%|██████    | 3/5 [03:00<02:03, 61.85s/it]

/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch1_mip/WT_pCenTy_001.tiff
(2160, 2560)


 80%|████████  | 4/5 [04:10<01:04, 64.88s/it]

/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/ch1_mip/Delta_adk1_pCenTy_002.tiff
(2160, 2560)


100%|██████████| 5/5 [05:18<00:00, 63.79s/it]


## 5) SPOT DETECTION

In [6]:
def detection_without_segmentation(
                            rna,
                            sigma,
                            min_distance = [3,3, 3],
                            threshold = None):


    rna_log = stack.log_filter(rna, sigma)
    mask = detection.local_maximum_detection(rna_log, min_distance=min_distance)

    if threshold is None:
        threshold = detection.automated_threshold_setting(rna_log, mask)
    spots, _ = detection.spots_thresholding(rna_log, mask, threshold)
    return spots

def detection_folder(
                     round_folder_path = "/media/tom/T7/Stitch/acquisition/",
                     round_name_regex = "r",
                     image_name_regex = "opool1_1_MMStack_3",
                     channel_name_regex = "ch1",
                     fixed_round_name = "r1_bc1",
                     path_output_segmentaton = "/media/tom/T7/stich0504/segmentation_mask/",
                      min_distance=(4, 4, 4),
                      scale_xy=0.103,
                      scale_z=0.300,
                      sigma = 1.35,
                        ### detection parameters with segmentation
                    dico_translation = np.load("/media/tom/T7/Stitch/acquisition/dico_translation.npy", allow_pickle=True).item(),
                    diam_um=20,
                    local_detection = True,
                    min_cos_tetha=0.75,
                    order=5,
                    test_mode=False,
                    threshold_merge_limit= 0.330):
    """

    :param sigma:
    :param rna_path:
    :param path_output_segmentaton:
    :param threshold_input:
    :param output_file:
    :param min_distance:
    :param local_detection:
    :param diam:
    :param scale_xy:
    :param scale_z:
    :param min_cos_tetha:
    :param order:
    :param test_mode:
    :return:


    """

    dico_spots = {}
    dico_intensity = {}
    for path_round in tqdm(list(Path(round_folder_path).glob(f"{round_name_regex}*"))[4:] + list(Path(round_folder_path).glob(f"{round_name_regex}*"))):
        print()
        print(path_round.name)
        dico_spots[path_round.name] = {}
        for path_rna_img in tqdm(list(path_round.glob(f"*{channel_name_regex}*ti*"))):

            if image_name_regex not in path_rna_img.name:
                continue
            print(path_rna_img.name)

            rna_img = tifffile.imread(path_rna_img)

            
            all_spots = detection_without_segmentation(
                            rna=rna_img,
                            sigma=sigma,
                            min_distance=min_distance,
                            threshold = None
                )
            dico_spots[path_round.name][path_rna_img.name] = all_spots
            
    return dico_spots


In [10]:

round_folder_path  = "/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/round_folder/"

dico_spots = detection_folder(
    round_folder_path=round_folder_path,
    round_name_regex="ch",
    image_name_regex="ti",
    channel_name_regex=".",
    min_distance=(3, 3),
    sigma=1,
    fixed_round_name=None,
    path_output_segmentaton=None,
    scale_xy=None,
    scale_z=None,
    ### detection parameters with segmentation
    dico_translation=None,
    diam_um=None,
    local_detection=None,
    min_cos_tetha=None,
    order=None,
    test_mode=None,
    threshold_merge_limit=None)


#### dico spot to csv

np.save(round_folder_path+"dico_spots.npy", dico_spots)
Path(round_folder_path+"detection_csv/").mkdir(parents=True, exist_ok=True)
for round_name in dico_spots.keys():
    for image_name in dico_spots[round_name].keys():
        df = pd.DataFrame(dico_spots[round_name][image_name])
        df = df.rename(columns={0: "x", 1: "y"})
        df.to_csv(f"{round_folder_path}detection_csv/{image_name}.csv",
                  index=False)

  0%|          | 0/1 [00:00<?, ?it/s]


ch0_mip



  0%|          | 0/5 [00:00<?, ?it/s]

WT_pCenTy_002.tiff



 20%|██        | 1/5 [00:00<00:03,  1.05it/s]

...


 60%|██████    | 3/5 [00:02<00:01,  1.33it/s]

WT_pCenTy_001.tiff



 80%|████████  | 4/5 [00:03<00:00,  1.29it/s]

Delta_adk1_pCenTy_002.tiff



100%|██████████| 1/1 [00:03<00:00,  3.79s/it]


# Count spots per nucleus and intensity

In [12]:



path_main_folder  = "/home/tom/Bureau/phd/Batrin/Images_pour_FISH/Expérience2/"
path_ch0 = path_main_folder + "round_folder/ch0_mip/"
path_mask_folder  = path_main_folder + "ch1_mask_mip/20_cyto/"
path_csv_folder = path_main_folder + "round_folder/detection_csv/"
dico_mean_spots = {}

dico_snr = {}
dico_mean_intensity = {}
dico_mean_intensity_normalize = {}


for path_mask in Path(path_mask_folder).glob("*tif*"):
    print(path_mask.name)
    mask = tifffile.imread(path_mask)
    img_fish = tifffile.imread(path_ch0 + path_mask.name[:-4])
    img_fish_mean = stack.mean_filter(img_fish, kernel_shape = "square", kernel_size = 4)


    df_spots = pd.read_csv(path_csv_folder + path_mask.name[:-4] + ".csv")
    df_spots['in_mask'] = 0
    df_spots['mean_intensity'] = 0
    df_spots['normalized_mean_intensity'] = 0

    list_spots = np.array([[x,y] for x, y in zip(df_spots['x'], df_spots['y'])])
    snr  = detection.compute_snr_spots(img_fish, list_spots, voxel_size = 108, spot_radius= 300)
    dico_snr[path_mask.name[:-4]] = snr
    list_mean_intensity = []
    mean_norm = np.mean(img_fish_mean[mask == 0])
    for i in range(len(df_spots)):
        if mask[int(df_spots['x'][i]), int(df_spots['y'][i])]  !=0:
            df_spots['in_mask'][i] =  mask[int(df_spots['x'][i]), int(df_spots['y'][i])]
            list_mean_intensity.append(img_fish_mean[int(df_spots['x'][i]), int(df_spots['y'][i])])


        df_spots['mean_intensity'][i] = img_fish_mean[int(df_spots['x'][i]), int(df_spots['y'][i])]
        df_spots['normalized_mean_intensity'][i] = img_fish_mean[int(df_spots['x'][i]), int(df_spots['y'][i])] / mean_norm
    df_spots.to_csv(path_csv_folder + path_mask.name[:-4] + ".csv", index=False)

    dico_mean_spots[path_mask.name[:-4]] = np.sum(df_spots['in_mask'] != 0) / len(np.unique(mask)[1:])

    dico_mean_intensity[path_mask.name[:-4]] = np.mean(list_mean_intensity)

    dico_mean_intensity_normalize[path_mask.name[:-4]] = np.mean(list_mean_intensity) / np.mean(img_fish_mean[mask == 0])



Delta_adk1_pCenTy_003.tiff.tif
WT_pCenTy_002.tiff.tif
WT_pCenTy_001.tiff.tif
Delta_adk1_pCenTy_002.tiff.tif
Delta_adk1_pCenTy_001.tiff.tif
